<font size='10'><h1 align="center"><font color='blue'>iPy oxDNA Examples</font></h1>

<center>
<img src="oxDNA.png">
</center>

<font size='10'><a class="anchor" id="3"></a> <h2 align="center"> <font color='black'>Duplex Melting</font></h2> 

In [ ]:
from ipy_oxdna.umbrella_sampling import ComUmbrellaSampling, MeltingUmbrellaSampling
from ipy_oxdna.oxdna_simulation import SimulationManager, Simulation, Observable
from ipy_oxdna.wham_analysis import collect_coms
import os
import matplotlib.pyplot as plt
from vmmc import *
from collections import Counter
from ipy_oxdna.parameter_search import *
import numpy as np
from skopt import Optimizer
from skopt.space import Categorical, Space
%load_ext autoreload
%autoreload 2

In [ ]:
path = os.path.abspath('/scratch/mlsample/ipy_oxDNA/ipy_oxdna_examples/duplex_melting/multi_system_runs')
system = f'{path}/duplex_50'
us = MeltingUmbrellaSampling(path, system)


In [ ]:
hist = us.continuous_to_discrete_unbiasing(8)

In [ ]:
hb_values_array = [self.hb_by_window[idx].values.T[0] for idx in range(self.n_windows)]

bins = list(np.linspace(0, max_hb, max_hb +1, dtype=int))
counts_i_b = [np.bincount(hb_values, minlength=max_hb+1) for hb_values in hb_values_array]
numerator = np.sum(counts_i_b, axis=0)

bias = [value.values.T[0] for value in self.umbrella_bias]
F_i = [float(F) for F in self.get_biases()]
beta = 1 / self.temperature
denominator = np.array([sum(counts) * np.exp((F - w) * beta) for counts, F, w in zip(counts_i_b, F_i, bias)])
   
unbiased_discrete_window = np.array([np.zeros(max_hb + 1) for _ in range(self.n_windows)]
for idx in range(self.n_windows):
    np.add.at(unbiased_discrete_window[idx], hb_values_array[idx], denominator[idx])
    
denominator = np.sum(unbiased_discrete_window, axis=0)

In [ ]:
true = hist[0] / hist[1]

normed = true / sum(true)

plt.plot(range(9), normed)

In [ ]:
us.calculate_melting_temperature_using_vmmc()

In [ ]:
plt.plot(range(9), us.free_energy_discrete)

In [ ]:
us.free

In [ ]:
us.plot_free()

In [ ]:
us.get_com_distance_by_window()
us.get_bias_potential_value(us.wham.xmin, us.wham.xmax, us.n_windows, us.wham.umbrella_stiff)
us.get_r0_values()

In [ ]:
beta = 1/us.temperature
beta

In [ ]:
F_i = np.array([float(F) for F in us.get_biases()])
bins = np.linspace(us.wham.xmin, us.wham.xmax, 200)
temp = [np.histogram(com_values, bins=bins) for com_values in us.com_by_window.values()]
p_i_b = np.array([value[0] for value in temp])

In [ ]:
numerator = np.sum(p_i_b, axis=0)

In [ ]:
denominator = np.array([sum(counts) * np.exp((F - w) * beta) for counts, F, w in zip(p_i_b, F_i, window_biases)])

In [ ]:
p_x = numerator / np.sum(denominator, axis=0)[1:]

In [ ]:
def w_i(r, r0, stiff):
    return 0.5 * float(stiff) * (r - r0)**2
window_biases = np.array([np.array([w_i(bin_value, r0_value, us.wham.umbrella_stiff) for bin_value in bins], dtype=np.longdouble) for r0_value in us.r0])

In [ ]:
plt.plot(bins[1:], -np.log(p_x))

In [ ]:
bias = np.array([value.values.T[0] for value in us.umbrella_bias], dtype=np.longdouble)
bias.shape

In [ ]:
bias_term = np.exp(beta * bias, dtype=np.longdouble)
bias_term.shape

In [ ]:
ensemble_bias_numerator = np.sum(np.exp(-beta * bias_term), axis=1)
ensemble_bias_denominator = np.full((bias_term.shape[0]), bias_term.shape[1])

ensemble_bias_term = ensemble_bias_numerator / ensemble_bias_denominator

                                    

In [ ]:
weighted_bias_term = np.array([bias * ensemble_bias for bias, ensemble_bias in zip(bias_term, ensemble_bias_term)])

In [ ]:
a = [value.values.T[0] for value in list(us.com_by_window.values())]


In [ ]:
weights = [1 / w_bias for w_bias in weighted_bias_term]
weights[0].shape

In [ ]:
bins = np.linspace(0, us.wham.xmax, 2000)
bins

In [ ]:
unbiased_counts = [np.histogram(com, bins=bins, weights=weight) for com, weight in zip(a, weights)]
unbiased_hists = [count[0] / sum(count[0]) for count in unbiased_counts]

In [ ]:
for hist in unbiased_hists:
    plt.plot(bins[1:], hist)

In [ ]:
counts = np.array([sum(count[0]) for count in unbiased_counts])
counts = np.full((100), len(weights[0]))
counts

In [ ]:
a_i = np.array([count * np.exp((-beta * bi) + (beta * F)) for count, bi, F in zip(counts, bias, F_i)])

In [ ]:
p_i = [a / sum(a) for a in a_i]
p_i

In [ ]:
window_weight = np.mean(a_i, axis=1) / np.sum(np.mean(a_i, axis=1))
weighted_hist = [hists[0] * weight for hists, weight in zip(unbiased_counts, window_weight)]
window_weight

In [ ]:
combined = np.sum(weighted_hist, axis=0)
combined.shape

In [ ]:
plt.plot(bins[1:], combined)

In [ ]:
for hist in weighted_hist:
    plt.plot(bins[1:], hist)

In [ ]:
bin_width = bins[2] - bins[1]
P_i_b = [p_i / sum(p_i) for p_i in p_i_b]

In [ ]:
plt.figure()
for p_b in P_i_b:
    plt.plot(bins[1:], p_b)

In [ ]:
window_biases.shape

In [ ]:
print(len(P_i_b))
print(len(window_biases))
print(len(ensemble_weight))

In [ ]:

ensemble_weight = [sum(np.exp(-beta * w)) / len(w) for w in window_biases]
weight_per_bin = [np.exp(beta  * w) for w in window_biases]
weigted_bias = [weight * ensemble for weight, ensemble in zip(weight_per_bin, ensemble_weight)]

P_i_u = [P_i * bias[1:] / sum(P_i * bias[1:]) for P_i, bias in zip(P_i_b, weigted_bias)]


In [ ]:
samples_per_window = [len(window) for window in us.com_by_window.values()][0]
F_i = [float(F) for F in F_i]
a_i = [samples_per_window * np.exp((-beta * w) + (beta * F)) for w, F in zip(window_biases, F_i)]

In [ ]:
p_i = a_i / sum(a_i)

In [ ]:
P_u = np.sum([p[1:] * P_i for p, P_i in zip(p_i, P_i_u)], axis=0)


In [ ]:
plt.plot(bins[1:], P_u)

In [ ]:
#Define umbrella sampling param space
param_space = Space([
    Categorical([0.5, 1, 5, 10], name='k_value'),
    Categorical([25, 40, 52, 70], name='temperature'),
    Categorical([True, False], name='unique_binding'),
    Categorical([1, 10, 100, 1000], name='print_every'),
    Categorical([5, 10, 15, 20], name='xmax'),
    Categorical([1e8, 5e8], name='production_step'),
    Categorical([20, 50], name='n_window')
])

inital_parameters = [
 ['10.0', '25', 'False', '10', '5', '500000000.0', '50'],
 ['0.5', '25', 'False', '100', '10', '500000000.0', '50'],
 ['10.0', '70', 'True', '10', '20', '500000000.0', '20']
]
    
#Define ground truth melting
path = os.path.abspath('/scratch/mlsample/ipy_oxDNA/ipy_oxdna_examples')
systems = ['duplex_melting']

file_dir_list = [f'{path}/{sys}' for sys in systems]
sim_dir_list = [f'{file_dir}/vmmc_melting_replicas/vmmc_melting_rep' for sys, file_dir in zip(systems, file_dir_list)]

n_replicas = 40
vmmc_replica_generator = VmmcReplicas()

vmmc_replica_generator.multisystem_replica(
    systems,
    n_replicas,
    file_dir_list,
    sim_dir_list
)
vmmc_sim_list = vmmc_replica_generator.sim_list
queue_of_simulations = vmmc_replica_generator.queue_of_sims

vmmc_sim_list[0].analysis.read_vmmc_op_data()
vmmc_sim_list[0].analysis.calculate_sampling_and_probabilities()
vmmc_sim_list[0].analysis.calculate_and_estimate_melting_profiles()
   
#Define system parameters
path = os.path.abspath('/scratch/mlsample/ipy_oxDNA/ipy_oxdna_examples/duplex_melting/parameter_scan')
max_hb = 8
com_list = '8,9,10,11,12,13,14,15'
ref_list = '0,1,2,3,4,5,6,7'
vmmc_ground_truth = vmmc_sim_list[0]

#Run
n_iterations = 5
batch_size = 3
run_baysian_hyperparameter_optimization(
    param_space,
    vmmc_ground_truth,
    max_hb, path,
    com_list,
    ref_list,
    n_iterations,
    batch_size,
    resume_state=None,
    inital_parameters=inital_parameters,
    subprocess=False
)

In [ ]:
us = MeltingUmbrellaSampling(file_dir, systems[2])

In [ ]:
us_list[0].continuous_to_discrete_unbiasing(8)

In [ ]:
plt.plot(1 / np.exp(-us_list[0].umbrella_bias[-1] / us_list[0].temperature), rasterized=True)

In [ ]:
a = us.unbiased_discrete_window
b = np.sum(us.unbiased_discrete_window, axis=1, keepdims=True)
normed = np.divide(a, b, out=np.zeros_like(a), where=b!=0)
a

In [ ]:
plt.plot(range(9), us_list[0].free_energy_discrete)

In [ ]:
path = os.path.abspath('/scratch/mlsample/ipy_oxDNA/ipy_oxdna_examples/duplex_melting/parameter_scan')
systems = ['_'.join(params) for params in inital_parameters]
file_dir = path

us_list = [MeltingUmbrellaSampling(file_dir, system) for system in systems[:2]]

In [ ]:
us.calculate_melting_temperature_using_vmmc()

<font size='10'><a class="anchor" id="3"></a> <h2 align="center"> <font color='black'>Potential Energy Umbrella Temperature Interpolation</font></h2> 

In [ ]:
string ="""
10.0_25_False_10_5_500000000.0_50
0.5_25_False_100_10_500000000.0_50
10.0_70_True_10_20_500000000.0_20
""".strip().split('\n')
inital_parameters = []
for st in string:
    inital_parameters.append(st.split('_'))
inital_parameters

In [ ]:
path = os.path.abspath('/scratch/mlsample/ipy_oxDNA/ipy_oxdna_examples/duplex_melting/')

system_name = 'multi_system_runs'

# k_value_list = ['5_normal_1', '5_unique_1']
k_value_list = ['10']

systems = [f'duplex_52_{stiff}' for stiff in k_value_list]

file_dirs = [f'{path}/{system_name}' for _ in range(len(systems))]


com_list = '8,9,10,11,12,13,14,15'
ref_list = '0,1,2,3,4,5,6,7'
xmin = 0
xmax = 10
n_windows = 100
starting_r0 = 1
steps = 1e6
# stiff = 0.4


pre_equlibration_parameters_list = [{'backend':'CPU','steps':'1e6','print_energy_every': '4e7',
                           'print_conf_interval':'4e7', "CUDA_list": "no",
                           'use_edge': 'false', 'refresh_vel': '1',
                           'fix_diffusion': '0', 'fix_diffusion_every': '1000',
                           'T':f'52C'} for _ in k_value_list]

equlibration_parameters_list = [{'backend':'CPU','steps':'5e7','print_energy_every': '4e7',
                           'print_conf_interval':'4e7', "CUDA_list": "no",
                           'use_edge': 'false', 'refresh_vel': '1',
                           'fix_diffusion': '0', 'fix_diffusion_every': '1000',
                           'T':f'52C'} for _ in k_value_list]

production_parameters_list = [{'backend':'CPU','steps':'1e8','print_energy_every': '1e8',
                        'print_conf_interval':'1e8', "CUDA_list": "no",
                        'use_edge': 'false', 'refresh_vel': '1',
                        'fix_diffusion': '0', 'max_io':'100', 'fix_diffusion_every': '1000',
                        'T':f'52C'} for temp in k_value_list]

us_list = [MeltingUmbrellaSampling(file_dir, sys, clean_build='force') for file_dir, sys in zip(file_dirs,systems)]

simulation_manager = SimulationManager()

In [ ]:
for us, pre_equlibration_parameters, stiff in zip(us_list, pre_equlibration_parameters_list, k_value_list):
    print(us.system)
    us.build_pre_equlibration_runs(simulation_manager, n_windows, com_list, ref_list,
                               stiff, xmin, xmax, pre_equlibration_parameters, starting_r0, steps,
                               print_every=1000, observable=True, protein=None,
                               force_file=None, continue_run=False)

In [ ]:
simulation_manager.worker_manager(cpu_run=True, gpu_mem_block=False)

In [ ]:
for us, equlibration_parameters, stiff in zip(us_list, equlibration_parameters_list, k_value_list):
    print(us.system)
    us.build_equlibration_runs(simulation_manager, n_windows, com_list, ref_list,
                               stiff, xmin, xmax, equlibration_parameters,
                               print_every=1000, observable=True, protein=None,
                               force_file=None, continue_run=False)

In [ ]:
for us in us_list[1:]:
    us.modify_topology_for_unique_pairing()

In [ ]:
simulation_manager.worker_manager(cpu_run=True, gpu_mem_block=False)

In [ ]:
for us, production_parameters, stiff in zip(us_list, production_parameters_list, k_value_list):
    print(us.system)
    us.build_production_runs(simulation_manager, n_windows, com_list, ref_list,
                             stiff, xmin, xmax, production_parameters,
                             observable=True, print_every=1e1 ,protein=None,
                             force_file=None, continue_run=False)

In [ ]:
simulation_manager.run(cpu_run=True, gpu_mem_block=False)

In [ ]:
for us in us_list:
    us.com_distance_observable(com_list, ref_list)
    us.hb_list_observable(print_every=1)


for us in us_list:
    plt.figure(dpi=200)
    for idx in range(0,100,1):
        us.analysis.view_observable('prod', idx, observable=us.observables_list[1], sliding_window=False)
    # plt.legend([us.system], fontsize=7)

In [ ]:
fig, ax = plt.subplots()

vmmc_sim_list[1].analysis.plot_weighted_histogram(n_bins=1000, label='vmmc 52C', ax=ax)
for us in us_list:
    us.wham.plot_free_energy(ax = ax)
plt.legend()
plt.xlim(-1, 13)
plt.ylim(-0.5,10)

In [ ]:
vmmc_sim_list[1].analysis.read_vmmc_op_data()
vmmc_sim_list[1].analysis.calculate_sampling_and_probabilities()


In [ ]:
us_list[0].read_potential_energy()

In [ ]:
e_state = [states.sum(axis=1) for states in us_list[0].potential_energy_by_window.values()]
e_state

In [ ]:
#Constants
STCK_FACT_EPS_OXDNA2 = 2.6717
STCK_BASE_EPS_OXDNA2 = 1.3523

# Initialize the variables
old_temperature = us_list[0].temperature

temperature_range = np.array((range(30, 70, 2)))
new_temperatures = (temperature_range + 273.15) / 3000
e_state = [states.sum(axis=1) for states in us_list[0].potential_energy_by_window.values()]
e_stack = [states['stacking'] for states in us_list[0].potential_energy_by_window.values()]
e_ext = 0  # Replace with actual value (external energy term)
am = 1  # Replace with actual value (the increment amount)
w = 0  # Replace with actual value (the weight)

# Initialize a list to store the results
results_list = []

for window_idx, (e_state_window, e_stack_window) in enumerate(zip(e_state, e_stack)):
    n_data_points = len(e_state_window)
    n_temps = len(new_temperatures)

    # Initialize a NumPy array for this window
    results_window = np.zeros((n_temps, n_data_points))

    # Calculate et for the old temperature (simtemp in C++)
    et_old = (e_stack_window * STCK_FACT_EPS_OXDNA2) / (STCK_BASE_EPS_OXDNA2 + old_temperature * STCK_FACT_EPS_OXDNA2)

    # Calculate e0 for the old temperature
    e0_old = e_state_window - old_temperature * et_old

    for temp_idx, new_temp in enumerate(new_temperatures):
        # Calculate et for the new temperature
        et_new = (e_stack_window * STCK_FACT_EPS_OXDNA2) / (STCK_BASE_EPS_OXDNA2 + new_temp * STCK_FACT_EPS_OXDNA2)

        # Calculate e0 for the new temperature
        e0_new = e_state_window - new_temp * et_new

        # Calculate the expression inside exp() for the new temperature
        energy_term = -(e0_old + e_ext + new_temp * et_new) / new_temp + (e0_old + e_ext + old_temperature * et_old) / old_temperature

        # Update results_window
        results_window[temp_idx, :] = am * np.exp(energy_term.values)

    # Add results_window to results_list
    results_list.append(results_window)



In [ ]:
results_array[0][1]

In [ ]:
us_list[0].termperature_interpolation(8, temperature_range)

In [ ]:
bias = [
    [biases[0] / temp_bias
     for temp_bias in temperature_bias
    ] 
     for biases,temperature_bias in zip(bias, results_list)
]

In [ ]:
bias

In [ ]:
us_list[0].potential_energy_by_window[0].sum(axis=1)

In [ ]:
energy_bias_per_window_per_temperature, new_potential_energies, old_potential_energies = us_list[0]._calculate_energy_bias(temp_range)

In [ ]:
np.exp(old_potential_energies[0].sum(axis=1) / old_temperature - new_potential_energies[0][0] / new_temperatures[0])

In [ ]:
old_potential_energies[7]['stacking']

In [ ]:
old_epsilons = 1.3448 + 2.6568 * old_temperature
new_epsilon = 1.3448 + 2.6568 * new_temperatures

In [ ]:
old_epsilons / new_epsilon

In [ ]:
old_temperature = us_list[0].temperature
new_temperatures = (temp_range + 273.15) / 3000
energy_bias_per_window_per_temperature = [
    [np.exp( (old_potential.sum(axis=1) / old_temperature) - (new_pot / new_temp) )
     for new_pot, new_temp in zip(new_potential, new_temperatures)
    ]
    for old_potential, new_potential in zip(old_potential_energies, new_potential_energies)
]

In [ ]:
energy_bias_per_window_per_temperature[99]

In [ ]:
temp_range = np.array((range(30, 70, 2)))
us_list[0].termperature_interpolation(8, temp_range)

In [ ]:
us_list[0].prob_discrete

In [ ]:
plt.plot(range(9) ,vmmc_sim_list[1].statistics['wt_free'])
for temp,free in zip(temp_range, us_list[0].free_energy_discrete):
    plt.plot(range(9), free, label=f'{temp}')
plt.legend()


In [ ]:
vmmc_sim_list[1].statistics

In [ ]:
us_list[0].continuous_to_discrete_unbiasing(8)

In [ ]:
us_list[0].free_energy_discrete

In [ ]:
us_list[0].make_last_hist_files()

In [ ]:
us_list[0].run_wham_discete(8)

In [ ]:
us_list[0].homebrew_discrete_wham(8)

In [ ]:
plt.figure()
plt.plot(range(9) ,vmmc_sim_list[1].statistics['wt_free'])
plt.plot(range(9), us_list[0].free_energy_discrete)
plt.legend(['vmmc', 'us'])

In [ ]:
wham_dir = os.path.abspath('/scratch/mlsample/ipy_oxDNA/wham/wham')
n_bins = '200'
tol = '1e-5'
n_boot = '0'
for us, stiff in zip(us_list, k_value_list):
    us.wham_run(wham_dir, xmin, xmax, 5, n_bins, tol, n_boot)

In [ ]:
for file in us_list[0].production_sims[0].sim_files.file_list:
    if 'potential_energy.txt' in file:
        print(file)

In [ ]:
for us in us_list:
    us.read_potential_energy()

In [ ]:
us_list[0].potential_energy_by_window

In [ ]:
temperature_range = np.linspace(20, 70, 10)
_calculate_energy_bias

In [ ]:
import pyarrow
print(pyarrow.__version__)


<font size='10'><a class="anchor" id="3"></a> <h2 align="center"> <font color='black'>Potential Energy Observalble</font></h2> 

In [ ]:
path = os.path.abspath('/scratch/mlsample/ipy_oxDNA/ipy_oxdna_examples/duplex_melting/')

file_dir = f'{path}'
sim_dir = f'{file_dir}/potential_energy'

sim = Simulation(file_dir, sim_dir)


potential_energy_observable = Observable.potential_energy(print_every=100, split=True, name='potential_energy.txt')
simulation_manager = SimulationManager()

input_params = {
            "interaction_type": "DNA2",
            "salt_concentration": "0.1",
            "sim_type": "MD",
            "backend": "CPU",
            "backend_precision": "mixed",
            "use_edge": "1",
            "edge_n_forces": "1",
            "CUDA_list": "verlet",
            "CUDA_sort_every": "0",
            "max_density_multiplier": "3",
            "steps": "1e7",
            "ensemble": "nvt",
            "thermostat": "john",
            "T": "50C",
            "dt": "0.003",
            "verlet_skin": "0.5",
            "diff_coeff": "2.5",
            "newtonian_steps": "103",
            "lastconf_file": "last_conf.dat",
            "trajectory_file": "trajectory.dat",
            "refresh_vel": "1",
            "no_stdout_energy": "0",
            "restart_step_counter": "1",
            "energy_file": "energy.dat",
            "print_conf_interval": "1e4",
            "print_energy_every": "1e4",
            "time_scale": "linear",
            "max_io": "5",
            "external_forces": "0",
            "external_forces_file": "forces.json",
            "external_forces_as_JSON": "true"
            }

In [ ]:
sim.build(clean_build='force')
sim.add_observable(potential_energy_observable)
sim.input_file(input_params)

simulation_manager.queue_sim(sim)

In [ ]:
simulation_manager.run(cpu_run=True, gpu_mem_block=False)

<font size='10'><a class="anchor" id="3"></a> <h2 align="center"> <font color='black'>Parameter Scan</font></h2> 

In [ ]:


path = os.path.abspath('/scratch/mlsample/ipy_oxDNA/ipy_oxdna_examples')
systems = ['duplex_melting']

file_dir_list = [f'{path}/{sys}' for sys in systems]
sim_dir_list = [f'{file_dir}/vmmc_melting_replicas/vmmc_melting_rep' for sys, file_dir in zip(systems, file_dir_list)]

n_replicas = 40
vmmc_replica_generator = VmmcReplicas()

vmmc_replica_generator.multisystem_replica(
    systems,
    n_replicas,
    file_dir_list,
    sim_dir_list
)
vmmc_sim_list = vmmc_replica_generator.sim_list
queue_of_simulations = vmmc_replica_generator.queue_of_sims

vmmc_sim_list[0].analysis.read_vmmc_op_data()
vmmc_sim_list[0].analysis.calculate_sampling_and_probabilities()
vmmc_sim_list[0].analysis.calculate_and_estimate_melting_profiles()



In [ ]:
param_space = Space([
    Categorical([0.5, 1, 5, 10], name='k_value'),
    Categorical([25, 50, 70], name='temperature'),
    Categorical([True, False], name='unique_binding'),
    Categorical([1, 10, 100, 1000], name='print_every'),
    Categorical([5, 10, 15, 20], name='xmax'),
    Categorical([1e8, 5e8], name='production_step'),
    Categorical([20, 50], name='n_window')
])
path = os.path.abspath('/scratch/mlsample/ipy_oxDNA/ipy_oxdna_examples/duplex_melting/parameter_optimization')
max_hb = 8
com_list = '8,9,10,11,12,13,14,15'
ref_list = '0,1,2,3,4,5,6,7'
vmmc_ground_truth = vmmc_sim_list[0]

In [ ]:
n_iterations = 3
batch_size = 3
run_baysian_hyperparameter_optimization(
    param_space,
    vmmc_ground_truth,
    max_hb, path,
    com_list,
    ref_list,
    n_iterations,
    batch_size,
    resume_state=None,
    subprocess=True
)

In [ ]:
vmmc_sim_list[0].analysis.sigmoid(samples[1], *vmmc_sim_list[0].analysis.popt)

In [ ]:
parameters = np.array([[1, 10], [50, 25], [True, False], [10, 100], [20, 10], [1e8, 1e8], [50, 50]], dtype='object')
ground_truth = vmmc_sim_list[0]


In [ ]:
run_baysian_hyperparameter_optimization

In [ ]:
spawn(objective_function, args=(ground_truth, max_hb,path, com_list, ref_list, *parameters))

In [ ]:
us.vmmc_sim.statistics

In [ ]:
vmmc_sim_list[0].analysis.read_vmmc_op_data()

In [ ]:
vmmc_sim_list[0].analysis.plot_statistics()

In [ ]:
os.getcwd()

In [ ]:
files = [file for file in os.listdir(path) if 'optimizer' in file]
which_number = sorted([int(file.split('.')[0].split('_')[-1]) for file in files])[-1]
for file in files:
    split = int(file.split('.')[0].split('_')[-1])
    if which_number == split:
        best_optim = [file]
best_optim[0]

<font size='10'><a class="anchor" id="3"></a> <h2 align="center"> <font color='black'>K value scan</font></h2> 

In [ ]:
path = os.path.abspath('/scratch/mlsample/ipy_oxDNA/ipy_oxdna_examples/duplex_melting/')

system_name = 'multi_system_runs'

k_value_list = ['0.1', '0.5', '0.75', '1', '2', '5', '10']
systems = [f'duplex_30_{stiff}' for stiff in k_value_list]

file_dirs = [f'{path}/{system_name}' for _ in range(len(systems))]


com_list = '8,9,10,11,12,13,14,15'
ref_list = '0,1,2,3,4,5,6,7'
xmin = 0
xmax = 15
n_windows = 50
starting_r0 = 1
steps = 1e6
stiff = 0.4


pre_equlibration_parameters_list = [{'backend':'CPU','steps':'1e6','print_energy_every': '4e7',
                           'print_conf_interval':'4e7', "CUDA_list": "no",
                           'use_edge': 'false', 'refresh_vel': '1',
                           'fix_diffusion': '0', 'fix_diffusion_every': '1000',
                           'T':f'30C'} for _ in k_value_list]

equlibration_parameters_list = [{'backend':'CPU','steps':'5e7','print_energy_every': '4e7',
                           'print_conf_interval':'4e7', "CUDA_list": "no",
                           'use_edge': 'false', 'refresh_vel': '1',
                           'fix_diffusion': '0', 'fix_diffusion_every': '1000',
                           'T':f'30C'} for _ in k_value_list]

production_parameters_list = [{'backend':'CPU','steps':'1e9','print_energy_every': '1e8',
                        'print_conf_interval':'1e8', "CUDA_list": "no",
                        'use_edge': 'false', 'refresh_vel': '1',
                        'fix_diffusion': '0', 'fix_diffusion_every': '1000',
                        'T':f'30C'} for temp in k_value_list]

us_list = [MeltingUmbrellaSampling(file_dir, sys, clean_build='force') for file_dir, sys in zip(file_dirs,systems)]

simulation_manager = SimulationManager()

In [ ]:
us_list.append(MeltingUmbrellaSampling(f'{path}/{system_name}', 'duplex_30', clean_build='force'))

In [ ]:
for us, pre_equlibration_parameters, stiff in zip(us_list, pre_equlibration_parameters_list, k_value_list):
    print(us.system)
    us.build_pre_equlibration_runs(simulation_manager, n_windows, com_list, ref_list,
                               stiff, xmin, xmax, pre_equlibration_parameters, starting_r0, steps,
                               print_every=100, observable=True, protein=None,
                               force_file=None, continue_run=False)

In [ ]:
simulation_manager.run(cpu_run=True, gpu_mem_block=False)

In [ ]:
for us, equlibration_parameters, stiff in zip(us_list, equlibration_parameters_list, k_value_list):
    print(us.system)
    us.build_equlibration_runs(simulation_manager, n_windows, com_list, ref_list,
                               stiff, xmin, xmax, equlibration_parameters,
                               print_every=100, observable=True, protein=None,
                               force_file=None, continue_run=False)

In [ ]:
simulation_manager.run(cpu_run=True, gpu_mem_block=False)

In [ ]:
for us, production_parameters, stiff in zip(us_list, production_parameters_list, k_value_list):
    print(us.system)
    us.build_production_runs(simulation_manager, n_windows, com_list, ref_list,
                             stiff, xmin, xmax, production_parameters,
                             observable=True, print_every=100 ,protein=None,
                             force_file=None, continue_run=False)

In [ ]:
simulation_manager.run(cpu_run=True, gpu_mem_block=False)

In [ ]:
wham_dir = os.path.abspath('/scratch/mlsample/ipy_oxDNA/wham/wham')
n_bins = '200'
tol = '1e-5'
n_boot = '0'
for us, stiff in zip(us_list, k_value_list):
    us.wham_run(wham_dir, xmin, xmax, stiff, n_bins, tol, n_boot)

In [ ]:
with plt.style.context(['science', 'no-latex', 'bright']): 

    ax = us_list[0].wham.plt_fig()
    for us in us_list:
        us.wham.plot_free_energy(ax=ax)
    
    plt.legend(fontsize=6)
    
plt.xlabel('Center of mass distance (nm)')


In [ ]:
n_chunks = 2
data_added_per_iteration = 3

wham_dir = os.path.abspath('/scratch/mlsample/ipy_oxDNA/wham/wham')
n_bins = '200'
tol = '1e-5'
n_boot = '0'


for us in us_list:
    us.wham.get_n_data_per_com_file()
    us.convergence_analysis(n_chunks, data_added_per_iteration, wham_dir, xmin, xmax, stiff, n_bins, tol, n_boot)

<font size='10'><a class="anchor" id="3"></a> <h2 align="center"> <font color='black'>K value scan plotting</font></h2> 

In [ ]:
for us in us_list:
    us.com_distance_observable(com_list, ref_list)


for us in us_list:
    plt.figure(dpi=200)
    for idx in range(0,50,1):
        us.analysis.view_observable('prod', idx, observable=us.observables_list[0], sliding_window=False)
    # plt.legend([us.system], fontsize=7)

<font size='10'><a class="anchor" id="3"></a> <h2 align="center"> <font color='black'>Duplex Melting Movie</font></h2> 

In [ ]:
path = os.path.abspath('/scratch/mlsample/ipy_oxDNA/ipy_oxdna_examples/duplex_melting/')

system_name = 'multi_system_runs'

systems = ['duplex_melting_movie']
temperature_list = [50]

file_dirs = [f'{path}/{system_name}' for _ in range(len(systems))]


com_list = '8,9,10,11,12,13,14,15'
ref_list = '0,1,2,3,4,5,6,7'
xmin = 0
xmax = 15
n_windows = 100

stiff = 0.4

equlibration_parameters_list = [{'backend':'CPU','steps':'1e6','print_energy_every': '1e3',
                           'print_conf_interval':'1e3', "CUDA_list": "no",
                           'use_edge': 'false', 'refresh_vel': '1',
                           'fix_diffusion': '0', 'fix_diffusion_every': '1000',
                           'T':f'{temp}C'} for temp in temperature_list]

production_parameters_list = [{'backend':'CPU','steps':'1e9','print_energy_every': '1e8',
                        'print_conf_interval':'1e8', "CUDA_list": "no",
                        'use_edge': 'false', 'refresh_vel': '1',
                        'fix_diffusion': '0', 'fix_diffusion_every': '1000',
                        'T':f'{temp}C'} for temp in temperature_list]

us_list = [MeltingUmbrellaSampling(file_dir, sys, clean_build='force') for file_dir, sys in zip(file_dirs,systems)]

simulation_manager = SimulationManager()

In [ ]:
for us, equlibration_parameters in zip(us_list, equlibration_parameters_list):
    print(us.system)
    us.build_equlibration_runs(simulation_manager, n_windows, com_list, ref_list,
                               stiff, xmin, xmax, equlibration_parameters,
                               print_every=1000, observable=True, protein=None,
                               force_file=None, continue_run=False)

simulation_manager.run(cpu_run=True, gpu_mem_block=False)



<font size='10'><a class="anchor" id="3"></a> <h2 align="center"> <font color='black'>Temp Scan</font></h2> 

In [ ]:
path = os.path.abspath('/scratch/mlsample/ipy_oxDNA/ipy_oxdna_examples/duplex_melting/')

system_name = 'multi_system_runs'

systems = ['duplex_25', 'duplex_30', 'duplex_35', 'duplex_40', 'duplex_45', 'duplex_50', 'duplex_55', 'duplex_60', 'duplex_65', 'duplex_70']
temperature_list = [25, 30, 35, 40, 45, 50, 55, 60, 65, 70]

file_dirs = [f'{path}/{system_name}' for _ in range(len(systems))]


com_list = '8,9,10,11,12,13,14,15'
ref_list = '0,1,2,3,4,5,6,7'
xmin = 0
xmax = 15
n_windows = 100

stiff = 0.4

equlibration_parameters_list = [{'backend':'CPU','steps':'5e7','print_energy_every': '4e7',
                           'print_conf_interval':'4e7', "CUDA_list": "no",
                           'use_edge': 'false', 'refresh_vel': '1',
                           'fix_diffusion': '0', 'fix_diffusion_every': '1000',
                           'T':f'{temp}C'} for temp in temperature_list]

production_parameters_list = [{'backend':'CPU','steps':'1e9','print_energy_every': '1e8',
                        'print_conf_interval':'1e8', "CUDA_list": "no",
                        'use_edge': 'false', 'refresh_vel': '1',
                        'fix_diffusion': '0', 'fix_diffusion_every': '1000',
                        'T':f'{temp}C'} for temp in temperature_list]

us_list = [MeltingUmbrellaSampling(file_dir, sys, clean_build='force') for file_dir, sys in zip(file_dirs,systems)]

simulation_manager = SimulationManager()

<font size='10'><a class="anchor" id="3"></a> <h2 align="center"> <font color='black'>Temp scan convergence analysis</font></h2> 

In [ ]:
n_chunks = 2
data_added_per_iteration = 3

wham_dir = os.path.abspath('/scratch/mlsample/ipy_oxDNA/wham/wham')
n_bins = '200'
tol = '1e-5'
n_boot = '0'
xmin = 0
xmax = 15
n_windows = 100

stiff = 0.4

for us in us_list:
    us.wham.get_n_data_per_com_file()
    us.convergence_analysis(n_chunks, data_added_per_iteration, wham_dir, xmin, xmax, stiff, n_bins, tol, n_boot)

In [ ]:
for us in us_list:
    us.wham.plot_truncated_free_energy()
    plt.legend()
    plt.title(us.system)
    us.wham.plot_chunks_free_energy()
    plt.legend()
    plt.title(us.system)

In [ ]:
us_list.append(MeltingUmbrellaSampling(f'{path}', f'us_melting_52_no_non_canonical', clean_build='force'))

<font size='10'><a class="anchor" id="3"></a> <h2 align="center"> <font color='black'>Temp Scan plotting</font></h2> 

In [ ]:
for us in us_list:
    us.com_distance_observable(com_list, ref_list)


for us in us_list:
    plt.figure(dpi=200)
    for idx in range(0,100,1):
        us.analysis.view_observable('prod', idx, observable=us.observables_list[1], sliding_window=20)
    plt.legend([us.system], fontsize=7)

In [ ]:
for us in us_list:
    if us.free['Free'].iloc[4] != 0:
        us.free['Free'] = us.free['Free'].subtract(us.free['Free'].iloc[4])

In [ ]:
with plt.style.context(['science', 'no-latex', 'bright']): 

    ax = us_list[0].wham.plt_fig()
    for us in us_list:
        us.wham.plot_free_energy(ax=ax, errorevery=20)
    
    vmmc_replica_generator.plot_mean_free_energy_with_error_bars(num_bins=500, ax=ax, errorevery=20)
    # plt.legend(['52C umbrella all hydrogen bonds', '52C umbrella only intended bonds', '52C VMMC all hydrogen bonds (only biased to sample intended bonds)'] ,fontsize=6)
    
    plt.legend(fontsize=5)
    plt.xlabel('Center of mass distance (nm)')
    plt.xlim(-0.75,15)
    # plt.ylim(-0.25, 12)

In [ ]:
[[us_list[sys].production_sims[window].analysis.get_conf_count() for window in range(n_windows)]for sys in range(3)] 

In [ ]:
for us, equlibration_parameters in zip(us_list[-1:], equlibration_parameters_list[-1:]):
    print(us.system)
    us.build_equlibration_runs(simulation_manager, n_windows, com_list, ref_list,
                               stiff, xmin, xmax, equlibration_parameters,
                               print_every=1000, observable=True, protein=None,
                               force_file=None, continue_run=False)

simulation_manager.worker_manager(cpu_run=True, gpu_mem_block=False)



In [ ]:
for us, production_parameters in zip(us_list[-1:], production_parameters_list[-1:]):
    print(us.system)
    us.build_production_runs(simulation_manager, n_windows, com_list, ref_list,
                             stiff, xmin, xmax, production_parameters,
                             observable=True, print_every=1000 ,protein=None,
                             force_file=None, continue_run=False)

In [ ]:
simulation_manager.run(cpu_run=True, gpu_mem_block=False)


<font size='6'><a class="anchor" id="3"></a> <h2 align="center"> <font color='black'>Multisystem umbrella sampling</font></h2> 

In [ ]:
path = os.path.abspath('/scratch/mlsample/ipy_oxDNA/ipy_oxdna_examples/duplex_melting/')

system_name = 'multi_system_runs'

systems = ['print_100_normal_1e9', 'print_100_unique_1e9']
print_every_list = [100, 100]

file_dirs = [f'{path}/{system_name}' for _ in range(len(systems))]


com_list = '8,9,10,11,12,13,14,15'
ref_list = '0,1,2,3,4,5,6,7'
xmin = 0
xmax = 6
n_windows = 50
max_hb = 8
stiff = 0.8

equlibration_parameters = {'backend':'CPU','steps':'5e7','print_energy_every': '4e7',
                           'print_conf_interval':'4e7', "CUDA_list": "no",
                           'use_edge': 'false', 'refresh_vel': '1',
                           'fix_diffusion': '0', 'fix_diffusion_every': '1000',
                           'T':'52C'
                          }

production_parameters ={'backend':'CPU','steps':'1e9','print_energy_every': '1e8',
                        'print_conf_interval':'1e8', "CUDA_list": "no",
                        'use_edge': 'false', 'refresh_vel': '1',
                        'fix_diffusion': '0', 'fix_diffusion_every': '1000',
                        'T':'52C'
                        }

us_list = [MeltingUmbrellaSampling(file_dir, sys, clean_build='force') for file_dir, sys in zip(file_dirs,systems)]

simulation_manager = SimulationManager()



<font size='10'><a class="anchor" id="3"></a> <h2 align="center"> <font color='black'>Continuous to Discrete Unbiasing</font></h2> 

In [ ]:
max_hb = 8

In [ ]:
def count_division_normalize(arr):
    row_sums = np.sum(arr, axis=1, keepdims=True)
    return arr / row_sums

for us, stiff in zip(us_list[:-1], k_value_list):
    print(us.system)
    us.get_com_distance_by_window()
    us.get_hb_list_by_window()
    us.get_bias_potential_value(xmin, xmax, n_windows, stiff)
    
    for idx in us.hb_by_window.keys():
        hb_values = np.array(us.hb_by_window[idx].values.T[0])
        to_remove = np.where(hb_values > max_hb)[0]  # Indices where hb_values > max_hb
        if len(to_remove > 0):
        # Remove rows from hb_by_window DataFrame
            us.hb_by_window[idx].drop(index=to_remove, inplace=True)
            us.hb_by_window[idx].reset_index(drop=True, inplace=True)
            
            # Remove corresponding rows from umbrella_bias DataFrame
            us.umbrella_bias[idx].drop(index=to_remove, inplace=True)
            us.umbrella_bias[idx].reset_index(drop=True, inplace=True)
            
    unbiased_discrete_window = np.array([np.zeros(max_hb + 1) for _ in range(n_windows)])
    bias = np.array([1 / np.exp(-window /us.temperature) for window in us.umbrella_bias])
    print(us.temperature)
    index_to_add_at = np.array([np.array(us.hb_by_window[idx].values.T[0]) for idx in range(n_windows)])
    
    for idx in range(n_windows):
        np.add.at(unbiased_discrete_window[idx], index_to_add_at[idx], bias[idx][0])
  
    normalized_array = count_division_normalize(unbiased_discrete_window)
    new_array_v2 = np.zeros(unbiased_discrete_window.shape[1])
    
    # Iterate through each row and index
    for i, row in enumerate(normalized_array):
        new_array_v2 += row * unbiased_discrete_window[i]
    us.counts_discrete = new_array_v2
    # Normalize the new array using count division normalization
    new_array_sum_v2 = np.sum(new_array_v2)
    normalized_new_array_v2 = new_array_v2 / new_array_sum_v2
    us.free_energy_discrete = -np.log(normalized_new_array_v2)
    us.free_energy_discrete -= min(us.free_energy_discrete)
    us.prob_discrete = normalized_new_array_v2

In [ ]:
for us in us_list[:-1]:
    plt.plot(range(9), us.free_energy_discrete, label=us.system)
plt.legend()

In [ ]:
for us in us_list[:-1]:
    us.calculate_melting_temperature_using_vmmc()

In [ ]:
inverted_finfs = []
temperatures = []
for us in us_list[:-1]:
    inverted_finfs.append(1 - us.vmmc_sim.analysis.finfs[0])
    temperatures.append(float((us.system.split('_')[-1])))
# del inverted_finfs[2]
# del temperatures[2]
inverted_finfs

In [ ]:
def sigmoid(x, L, x0, k, b):
    return L / (1 + np.exp(-k * (x - x0))) + b

from scipy.optimize import curve_fit

p0 = [max(inverted_finfs), np.median(temperatures), 1, min(inverted_finfs)]  # initial guesses for L, x0, k, b
popt, _ = curve_fit(sigmoid, temperatures, inverted_finfs, p0, method='dogbox')

x_fit = np.linspace(min(temperatures), max(temperatures), 500)
y_fit = sigmoid(x_fit, *popt)

idx = np.argmin(np.abs(y_fit - 0.5))
Tm = x_fit[idx]

In [ ]:
# Plotting
# plt.figure(figsize=(10, 6))
vmmc_sim_list[0].analysis.plot_melting_profiles(label='VMMC Discrete ')

plt.scatter(temperatures, inverted_finfs, marker='o', label='Continous umbrella made discrete data')
plt.plot(x_fit, y_fit, linestyle='--', linewidth=2, label='Continous umbrella made discrete Sigmoid Fit')

# Add a vertical line at the melting temperature
plt.axvline(x=Tm, color='green', linestyle='--', linewidth=2, label=f'Continous umbrella made discrete Tm = {Tm:.2f} °C')

plt.xlabel('Temperature (°C)')
plt.ylabel('Fraction of ssDNA')
plt.title(f'Melting Profile')

# Set y-axis limits
plt.ylim(0, 1.1)

plt.legend()
plt.grid(True)
plt.show()


In [ ]:
plt.plot(inv_finfs.keys(), inv_finfs.values())

In [ ]:
# vmmc_sim_list[0].analysis.last_hist_analysis()
inv = 1 -  np.array(vmmc_sim_list[0].analysis.finfs)
inv

In [ ]:
us_list[1].vmmc_sim.statistics

In [ ]:
us.vmmc_sim.statistics

In [ ]:
free = vmmc_sim_list[0].statistics['wt_free'] - vmmc_sim_list[0].statistics['wt_free'].values[-1]
us.free_energy_discret us.free_energy_discrete - us.free_energy_discrete[-1]

In [ ]:
with plt.style.context(['science', 'no-latex', 'bright']): 
    ax = us_list[0].wham.plt_fig()
    for us in us_list:
        temp = us.free_energy_discrete - us.free_energy_discrete[-1]
        ax.plot(range(9), temp, label=us.system)
    ax.plot(range(9), free, label='discrete')

    plt.legend(fontsize=6)
    ax.set_xlabel('Number of Hydrogen Bonds')

In [ ]:
biases = us_list[1].umbrella_bias
shift = np.array(us_list[1].get_biases(), dtype=float)

energy = [np.exp(-bias + shift) for bias, shift in zip(biases, shift)]

In [ ]:
np.zeros(max_hb)

In [ ]:
us_list[1].unbias_com_to_hb(xmin, xmax, n_windows, stiff, max_hb)

In [ ]:
prob = us_list[1].run_wham_discete(8)

In [ ]:
prob = us_list[1].homebrew_discrete_wham(9)

In [ ]:
prob.tolist()

In [ ]:
inv = 1 -  np.array(vmmc_sim_list[0].analysis.finfs)
inv[9]

In [ ]:
vmmc_sim_list[0].analysis.temperatures.index(52)

In [ ]:
com_by_window = us_list[1].com_by_window
hb_by_window = us_list[1].hb_by_window
umbrella_bias = us_list[1].umbrella_bias
shift = np.array(us_list[1].get_biases(), dtype=float)

umbrella_bias = [np.exp(-bias) for bias, shift in zip(umbrella_bias, shift)]

In [ ]:
unbiased_discrete_window = {idx:np.zeros(int(max_hb + 1)) for idx in range(n_windows)}
for idx in range(n_windows):
    index_to_add_at = np.array(hb_by_window[idx].values.T[0])
    biases = np.array([value for value in umbrella_bias[idx].values.T[0]])
    value_to_add = 1 / biases
    np.add.at(unbiased_discrete_window[idx], index_to_add_at, value_to_add)
print(unbiased_discrete_window)

In [ ]:
unbiased = [hist) - s for hist,s in zip(unbiased_discrete_window.values(), shift)]

In [ ]:
unbiased

In [ ]:
sum_of_hist = np.zeros(9)
for hist in unbiased:
    sum_of_hist += hist
sum_of_hist /= sum(sum_of_hist)

In [ ]:
sum_of_hist.tolist()

In [ ]:
us.discrete_hist.values()

In [ ]:
plt.plot(range(9), -np.log(list(us.discrete_hist.values())))
plt.plot(range(9), -np.log(sum_of_hist))

In [ ]:
counters = [sum(hist) for hist in unbiased_discrete_window.values()]

In [ ]:
new_hist = [hist / count for hist,count in zip(unbiased_discrete_window.values(), counters)]

In [ ]:
new_hist

In [ ]:
us_list[0].homebrew_discrete_wham(9)

In [ ]:
max_hb = 8
plt.figure()
plt.plot(range(9), vmmc_sim_list[0].statistics['wt_free'], label='discrete')
plt.plot(range(9), -np.log(us_list[0].homebrew_discrete_wham(9)), label='here')

<font size='6'><a class="anchor" id="3"></a> <h2 align="center"> <font color='black'>Multisystem umbrella sampling</font></h2> 

In [ ]:
path = os.path.abspath('/scratch/mlsample/ipy_oxDNA/ipy_oxdna_examples/duplex_melting/')

system_name = 'multi_system_runs'

systems = ['print_100_normal_1e9', 'print_100_unique_1e9']
print_every_list = [100, 100]

file_dirs = [f'{path}/{system_name}' for _ in range(len(systems))]


com_list = '8,9,10,11,12,13,14,15'
ref_list = '0,1,2,3,4,5,6,7'
xmin = 0
xmax = 6
n_windows = 50

stiff = 0.8

equlibration_parameters = {'backend':'CPU','steps':'5e7','print_energy_every': '4e7',
                           'print_conf_interval':'4e7', "CUDA_list": "no",
                           'use_edge': 'false', 'refresh_vel': '1',
                           'fix_diffusion': '0', 'fix_diffusion_every': '1000',
                           'T':'52C'
                          }

production_parameters ={'backend':'CPU','steps':'1e9','print_energy_every': '1e8',
                        'print_conf_interval':'1e8', "CUDA_list": "no",
                        'use_edge': 'false', 'refresh_vel': '1',
                        'fix_diffusion': '0', 'fix_diffusion_every': '1000',
                        'T':'52C'
                        }

us_list = [MeltingUmbrellaSampling(file_dir, sys, clean_build='force') for file_dir, sys in zip(file_dirs,systems)]

simulation_manager = SimulationManager()

In [ ]:
for us, print_every in zip(us_list, print_every_list):
    us.build_equlibration_runs(simulation_manager, n_windows, com_list, ref_list,
                               stiff, xmin, xmax, equlibration_parameters,
                               print_every=print_every, observable=True, protein=None,
                               force_file=None, continue_run=False)


In [ ]:
us_list[1].modify_topology_for_unique_pairing()

In [ ]:
simulation_manager.run(cpu_run=True, gpu_mem_block=False)

In [ ]:
for us, print_every in zip(us_list, print_every_list):
    us.build_production_runs(simulation_manager, n_windows, com_list, ref_list,
                             stiff, xmin, xmax, production_parameters,
                             observable=True, print_every=print_every,protein=None,
                             force_file=None, continue_run=False)


In [ ]:
simulation_manager.run(cpu_run=True, gpu_mem_block=False)


In [ ]:
wham_dir = os.path.abspath('/scratch/mlsample/ipy_oxDNA/wham/wham')
n_bins = '200'
tol = '1e-5'
n_boot = '0'
for us in us_list:
    us.wham_run(wham_dir, xmin, xmax, stiff, n_bins, tol, n_boot)

In [ ]:
with plt.style.context(['science', 'no-latex', 'bright']): 
    ax = us_list[0].wham.plt_fig()
    for us in us_list:
        us.wham.plot_free_energy(ax=ax)
    vmmc_replica_generator.plot_mean_free_energy_with_error_bars(num_bins=1000, ax=ax, confidence_level=0.95, label='VMMC discrete made continuous 52C')
    plt.legend(fontsize=6)
    plt.xlim(0, 6)
    plt.ylim(0, 8)

In [ ]:
for us in us_list:
    us.com_distance_observable(com_list, ref_list)


for us in us_list[:2]:
    plt.figure(dpi=200)
    for idx in range(0,100,1):
        us.analysis.view_observable('prod', idx, sliding_window=50, observable=us.observables_list[0])#, observable=us.observables_list[0], sliding_window=20)
    plt.legend([us.system], fontsize=7)

<font size='6'><a class="anchor" id="3"></a> <h2 align="center"> <font color='black'>Unbiased equlibration</font></h2> 

In [ ]:
simulation_manager = SimulationManager() 
simulation_manager.start_nvidia_cuda_mps_control(pipe='temp')

In [ ]:
path = os.path.abspath('/scratch/mlsample/ipy_oxDNA/ipy_oxdna_examples')
file_dir = f'{path}/duplex_melting'
sim_dir = f'{file_dir}/unbiased_no_non_canonical_bonds_1'

sim = Simulation(file_dir, sim_dir)

In [ ]:
sim.build(clean_build='force')
sim.input_file({'backend':'CPU', 'steps':'1e7','print_energy_every': '1e4', 'print_conf_interval':'1e4', "CUDA_list": "no", 'use_edge': 'false'})

In [ ]:
sim.modify_topology_for_unique_pairing()

In [ ]:
sim.oxpy_run.run(subprocess=True)

In [ ]:
sim.oxpy_run.process.terminate()

In [ ]:
sim.analysis.plot_energy()
sim.analysis.view_last()

<font size='7'><a class="anchor" id="3"></a> <h2 align="center"> <font color='black'>Initalize Vmmc Replicas</font></h2> 

In [ ]:
path = os.path.abspath('/scratch/mlsample/ipy_oxDNA/ipy_oxdna_examples')
systems = ['duplex_melting']

file_dir_list = [f'{path}/{sys}' for sys in systems]
sim_dir_list = [f'{file_dir}/vmmc_melting_replicas/vmmc_melting_rep' for sys, file_dir in zip(systems, file_dir_list)]

n_replicas = 40
vmmc_replica_generator = VmmcReplicas()

vmmc_replica_generator.multisystem_replica(
    systems,
    n_replicas,
    file_dir_list,
    sim_dir_list
)
vmmc_sim_list = vmmc_replica_generator.sim_list
queue_of_simulations = vmmc_replica_generator.queue_of_sims

p1 = '15,14,13,12,11,10,9,8'
p2 = '0,1,2,3,4,5,6,7'
pre_defined_weights = [8, 16204, 1882.94, 359.746, 52.5898, 15.0591, 7.21252, 2.2498, 2.89783]

sim_parameters = {'T':'52C', 'steps':'1e9','print_energy_every': '1e5','print_conf_interval':'1e6'}

In [ ]:
vmmc_sim_list[0].analysis.read_vmmc_op_data()
vmmc_sim_list[0].analysis.calculate_sampling_and_probabilities()

In [ ]:
with plt.style.context(['science', 'no-latex', 'bright']):
    ax = us.wham.plt_fig() 
    vmmc_replica_generator.plot_mean_free_energy_with_error_bars(num_bins=500, ax=ax, confidence_level=0.95, label='VMMC discrete made continuous 52C')
    # us_1.wham.plot_free_energy(confidence_level=0.99, ax=ax, errorevery=10, label='Continuous umbrella sampling 52C all bonds')
    us.wham.plot_free_energy(confidence_level=0.99, ax=ax, errorevery=10, label='Continuos umbrella sampling 52C only intended bonds')
    ax.set_xlim(-0.1,8)
    plt.ylim(-0.1,9)
plt.legend(fontsize=6)

In [ ]:
with plt.style.context(['science', 'no-latex', 'bright']):
    ax = us.wham.plt_fig() 
    # vmmc_replica_generator.plot_mean_free_energy_with_error_bars(num_bins=500, ax=ax, confidence_level=0.95, label='VMMC discrete made continuous 52C')
    us_1.wham.plot_free_energy(confidence_level=0.99, ax=ax, errorevery=10, label='Continuous umbrella sampling 52C all bonds')
    us.wham.plot_free_energy(confidence_level=0.99, ax=ax, errorevery=10, label='Continuos umbrella sampling 52C only intended bonds')
    # ax.set_xlim(-0.1,8)
    # plt.ylim(-0.1,9)
plt.legend(fontsize=6)

<font size='7'><a class="anchor" id="3"></a> <h2 align="center"> <font color='black'>Umbrella sampling melting</font></h2> 

In [ ]:
simulation_manager = SimulationManager()

path = os.path.abspath('/scratch/mlsample/ipy_oxDNA/ipy_oxdna_examples')
file_dir = f'{path}/duplex_melting'
system = 'us_melting_52_cpu'

com_list = '8,9,10,11,12,13,14,15'
ref_list = '0,1,2,3,4,5,6,7'
xmin = 0
xmax = 6
n_windows = 100

stiff = 0.8

equlibration_parameters = {'backend':'CPU','steps':'1e7','print_energy_every': '5e6',
                           'print_conf_interval':'5e6', "CUDA_list": "no",
                           'use_edge': 'false', 'refresh_vel': '1',
                           'fix_diffusion': '0', 'fix_diffusion_every': '1000',
                           'T':'52C'
                          }

production_parameters ={'backend':'CPU','steps':'2e9','print_energy_every': '2e7',
                        'print_conf_interval':'2e7', "CUDA_list": "no",
                        'use_edge': 'false', 'refresh_vel': '1',
                        'fix_diffusion': '0', 'fix_diffusion_every': '1000',
                        'T':'52C'
                        }

us_1 = MeltingUmbrellaSampling(file_dir, system, clean_build=True)


In [ ]:
simulation_manager = SimulationManager()

path = os.path.abspath('/scratch/mlsample/ipy_oxDNA/ipy_oxdna_examples')
file_dir = f'{path}/duplex_melting'
system = 'us_melting_52_no_non_canonical_10'

com_list = '8,9,10,11,12,13,14,15'
ref_list = '0,1,2,3,4,5,6,7'
xmin = 0
xmax = 6
n_windows = 100

stiff = 0.8

equlibration_parameters = {'backend':'CPU','steps':'1e7','print_energy_every': '5e6',
                           'print_conf_interval':'5e6', "CUDA_list": "no",
                           'use_edge': 'false', 'refresh_vel': '1',
                           'fix_diffusion': '0', 'fix_diffusion_every': '1000',
                           'T':'52C'
                          }

production_parameters ={'backend':'CPU','steps':'2e9','print_energy_every': '2e7',
                        'print_conf_interval':'2e7', "CUDA_list": "no",
                        'use_edge': 'false', 'refresh_vel': '1',
                        'fix_diffusion': '0', 'fix_diffusion_every': '1000',
                        'T':'52C'
                        }

us = MeltingUmbrellaSampling(file_dir, system, clean_build=True)


<font size='6'><a class="anchor" id="3"></a> <h2 align="center"> <font color='black'>Build and run</font></h2> 

In [ ]:
us.build_equlibration_runs(simulation_manager, n_windows, com_list, ref_list, stiff, xmin, xmax, equlibration_parameters,
                           observable=True, sequence_dependant=False, print_every=10, name='com_distance.txt',
                           continue_run=False)

In [ ]:
us.modify_topology_for_unique_pairing()

In [ ]:
simulation_manager.run(cpu_run=True, gpu_mem_block=False)

In [ ]:
us.build_production_runs(simulation_manager, n_windows, com_list, ref_list, stiff, xmin, xmax, production_parameters,
                           observable=True, sequence_dependant=False, print_every=10, name='com_distance.txt',
                        continue_run=False)

In [ ]:
simulation_manager.run(cpu_run=True, gpu_mem_block=False)

In [ ]:
dir(us.equlibration_sims[0].sim_files)

<font size='6'><a class="anchor" id="3"></a> <h2 align="center"> <font color='black'>Run time analysis methods</font></h2> 

In [ ]:
us.analysis.view_conf('eq', 7)

In [ ]:
us.com_distance_observable(com_list, ref_list, print_every=10)
us.hb_list_observable(print_every=10)
plt.figure(dpi=50)
for idx in range(0,100,1):
    us.analysis.hist_observable('prod', idx, observable=us.observables_list[0])#, sliding_window=False)#, bins=200)#)
# plt.legend(fontsize=4)

In [ ]:
for idx in range(0,48):
    print(idx)
    us.equlibration_sims[idx].sim_files.last_conf

In [ ]:
us.analysis.view_conf('eq', 13)

In [ ]:
wham_dir = os.path.abspath('/scratch/mlsample/ipy_oxDNA/wham/wham')
n_bins = '200'
tol = '1e-5'
n_boot = '0'
us.wham_run(wham_dir, xmin, xmax, stiff, n_bins, tol, n_boot)

In [ ]:
with plt.style.context(['science', 'no-latex', 'bright']):
    ax = us.wham.plt_fig() 
    vmmc_replica_generator.plot_mean_free_energy_with_error_bars(num_bins=500, ax=ax, confidence_level=0.95, label='VMMC discrete made continuous 52C')
    us.wham.plot_free_energy(confidence_level=0.99, ax=ax, errorevery=10, label='Continuos umbrella sampling 52C only intended bonds')
    # ax.set_xlim(-0.1,8)
    # plt.ylim(-0.1,9)
plt.legend(fontsize=6)

In [ ]:
us.free

<font size='8'><a class="anchor" id="3"></a> <h2 align="center"> <font color='black'>Post run reinitalization </font></h2> 

In [ ]:
path = os.path.abspath('/scratch/mlsample/ipy_oxDNA/ipy_oxdna_examples')
file_dir = f'{path}/duplex_melting'
systems = ['us_melting_10', 'us_melting', 'us_melting_30','us_melting_40', 'us_melting_62']

us_list = [MeltingUmbrellaSampling(file_dir, system) for system in systems]

In [ ]:
us_list[0].fig_ax()

for us in us_list:
    us.wham.plot_free_energy(ax=us_list[0].ax, errorevery=10)
plt.legend([f'{10 + 273}K', f'{20 + 273}K', f'{30 + 273}K', f'{40 + 273}K', f'{62 + 273}K'])
plt.title('8nt Duplex Melting Free Energy Profile')
plt.xlabel('COM Distance (nm)')

<font size='6'><a class="anchor" id="3"></a> <h2 align="center"> <font color='black'>Convergence analysis </font></h2> 

In [ ]:
n_data = 50000

n_chunks = 2
data_added_per_iteration = 3

wham_dir = os.path.abspath('/scratch/mlsample/ipy_oxDNA/wham/wham')
n_bins = '200'
tol = '1e-5'
n_boot = '0'
xmin = 0
xmax = 6
n_windows = 50

stiff = 0.8

for us in us_list:
    us.convergence_analysis(n_chunks, data_added_per_iteration, wham_dir, xmin, xmax, stiff, n_bins, tol, n_boot)

In [ ]:
us.wham.plot_truncated_free_energy()
plt.legend()
plt.title(us.system)
us.wham.plot_chunks_free_energy()
plt.legend()
plt.title(us.system)

In [ ]:
n_data = 50000

n_chunks = 2
data_added_per_iteration = 3

wham_dir = os.path.abspath('/scratch/mlsample/ipy_oxDNA/wham/wham')
n_bins = '200'
tol = '1e-5'
n_boot = '0'
xmin = 0
xmax = 8
n_windows = 95

stiff = 0.8

for us in us_list:
    if 'us_melting_62' in us.system:
        us.convergence_analysis(n_chunks, data_added_per_iteration, wham_dir, xmin, 5, stiff, n_bins, tol, n_boot)
    else:
        us.convergence_analysis(n_chunks, data_added_per_iteration, wham_dir, xmin, xmax, stiff, n_bins, tol, n_boot)

In [ ]:
for us in us_list:
    
    us.wham.plot_truncated_free_energy()
    plt.legend()
    plt.title(us.system)
    us.wham.plot_chunks_free_energy()
    plt.legend()
    plt.title(us.system)

<font size='6'><a class="anchor" id="3"></a> <h2 align="center"> <font color='black'>In progress: Com unbiasing </font></h2> 

In [ ]:
xmin = 0
xmax = 6
n_windows = 100
stiff = 0.8
max_hb = 8

us.unbias_com_to_hb(xmin, xmax, n_windows, stiff, max_hb)

In [ ]:
x_range = np.round(np.linspace(xmin, xmax, (n_windows + 1))[1:], 3)

1 / (0.5 * stiff * (x_range[0] - list(us.com_by_window.values())[0][0][0])**2)

In [ ]:
list(us.com_by_window.values())[0][0][0]

In [ ]:
us.get_bias_potential_value(xmin, xmax, n_windows, stiff)
weight = 1 / us.umbrella_bias[0]

In [ ]:
sum(weight.values)

In [ ]:
us.make_last_hist_files()

In [ ]:
normalized_bins = {}

discrete_bins = list(range(max_hb + 1))
for key, value in us.unbiased_discrete_windows.items():
    normalized_bins[key] = value / sum(value)

In [ ]:
normalized_bins

In [ ]:
hist = np.zeros(int(max_hb + 1))
for key, value in normalized_bins.items():
    hist += value
    
hist = hist / sum(hist)
plt.plot(discrete_bins, -np.log(hist))


In [ ]:
us.get_biases()

In [ ]:
us.run_wham_discete(max_hb)

In [ ]:
us.discrete_hist

In [ ]:
plt.plot(us.discrete_hist.keys(), -np.log(list(us.discrete_hist.values())), label='Continuous made discrete')
plt.plot(range(9), vmmc_sim_list[0].statistics['wt_free'], label='discrete')
# plt.plot(discrete_bins, -np.log(hist))
plt.legend()

In [ ]:
us.hb_by_window[0].shape[0]

In [ ]:
vmmc_sim_list[0].analysis.read_vmmc_op_data()

In [ ]:
vmmc_sim_list[0].analysis.calculate_sampling_and_probabilities()

In [ ]:
plt.plot(range(9), vmmc_sim_list[0].statistics['wt_free'])

In [ ]:
plt.plot(vmmc_sim_list[0].vmmc_df['h_bonds'], np.log(vmmc_sim_list[0].vmmc_df['wt_occ']))

In [ ]:
vmmc_sim_list[0].analysis.calculate_weighted_histogram()

In [ ]:
vmmc_sim_list[0].analysis.calculate_sampling_and_probabilities()

In [ ]:
import numpy as np
biases = np.array(us.get_biases())
# biases = [float(bias) / ((273 + 20) / 3000) for bias in biases]
free_shift = [float(bias) for bias in biases]
bins = np.linspace(0, 8, 200)
com_by_window = us.get_com_distance_by_window()
histograms = [np.histogram(coms, bins=bins) for coms in com_by_window.values()]

In [ ]:
com_by_window

In [ ]:
# Optimize the weighted property calculation function by vectorizing operations where possible
def calculate_weighted_property_optimized(com_distance_dfs, hb_list_dfs, metadata_df, F_i_values):
    weighted_property = {}
    # Constants
    K_B = 1  # Boltzmann constant in simulation units
    T = 1  # Temperature in simulation units
    
    # Reset the index of metadata DataFrame
    metadata_df = metadata_df.reset_index(drop=True)
    
    for i, (com_distance_df, hb_list_df) in enumerate(zip(com_distance_dfs.values(), hb_list_dfs.values())):
        # Reset the index for both DataFrames for consistent indexing
        com_distance_df = com_distance_df.reset_index(drop=True)
        hb_list_df = hb_list_df.reset_index(drop=True)
        
        # Fetch window-specific parameters
        k_i = metadata_df.loc[i, 'Spring']
        x_0_i = metadata_df.loc[i, 'Loc_win_min']
        F_i = F_i_values[i]
        
        # Vectorized calculation of the weighting factor (Equation 13 in the WHAM paper)
        weighting_factor = np.exp((F_i - 0.5 * k_i * (com_distance_df['Value'] - x_0_i)**2) / (K_B * T))
        
        # Vectorized calculation of the weighted property (T(x))
        hb_values = hb_list_df['Value']
        weighted_numerator = weighting_factor * com_distance_df['Value']
        weighted_denominator = weighting_factor
        
        for x, (numerator, denominator) in zip(hb_values, zip(weighted_numerator, weighted_denominator)):
            if x not in weighted_property:
                weighted_property[x] = {'numerator': 0.0, 'denominator': 0.0}
            weighted_property[x]['numerator'] += numerator
            weighted_property[x]['denominator'] += denominator
    
    # Calculate the final weighted property T(x) (Equation 14 in the WHAM paper)
    weighted_property_result = {x: props['numerator'] / props['denominator'] for x, props in weighted_property.items()}
    
    return weighted_property_result

# Re-run the weighted property calculation with the optimized function
weighted_property_result_optimized = calculate_weighted_property_optimized(all_com_distance_dfs, all_h_bonds_dfs, metadata_df, F_i_values_fixed)

weighted_property_result_optimized

In [ ]:
def get_bias_potential_value(com_by_window, xmin, xmax, n_windows, stiff):
    x_range = np.round(np.linspace(xmin, xmax, (n_windows + 1))[1:], 3)
    umbrella_bias = [0.5 * stiff * (com_values - eq_pos)**2 for com_values, eq_pos in zip(com_by_window.values(), x_range)]
    return umbrella_bias
                     
def weighted_average_time_series(biases, com_by_window, xmin, xmax, n_windows, stiff):
    bias_potential_values = get_bias_potential_value(com_by_window, xmin, xmax, n_windows, stiff)
    
    exp_term = [np.exp(bias - free) for bias, free in zip(bias_potential_values, free_shift)]
    
    denominator = sum([sum(term) for term in exp_term])
    
    numberator = sum([sum(time_series * term) for time_series, term in zip(com_by_window, exp_term)])
    
    
    return numberator / denominator
        
    
    
    
def wham_probability_equation(histograms, free_shift, com_by_window, xmin, xmax, n_windows, stiff):
    #get the histogram counts
    counts = [hist[0] for hist in histograms]
    #get hist bins
    bins = histograms[0][1]
    #compute the numerator
    sum_of_counts_over_all_windows = sum(counts)
    #
    umbrella_bias = get_bias_potential_value(com_by_window, xmin, xmax, n_windows, stiff)
    shift_minus_umbrella_bias = [ sum(np.exp(free - bias)) for free, bias in zip(free_shift, umbrella_bias)]
    print(len(sum_of_counts_over_all_windows))
    
    return None

wham_probability_equation(histograms, free_shift, com_by_window, xmin, xmax, n_windows, stiff)

In [ ]:
histograms = [np.histogram(coms, bins=bins) for coms in com_by_window.values()]

In [ ]:
histograms

In [ ]:
import numpy as np
biases = np.array(us.get_biases())
# biases = [float(bias) / ((273 + 20) / 3000) for bias in biases]
free_shift = [float(bias) for bias in biases]
bins = np.linspace(0, 8, 200)
com_by_window = us.get_com_distance_by_window()
histograms = [np.histogram(coms, bins=bins) for coms in com_by_window.values()]

In [ ]:
def get_window_bias_potential_value(bins, xmin, xmax, n_windows, stiff):
    x_range = np.round(np.linspace(xmin, xmax, (n_windows + 1))[1:], 3)
    bins = bins[1:]
    print(bins)
    print(x_range)
    diffrence = [com_values - eq_pos for com_values, eq_pos in zip(bins, x_range)]
    umbrella_bias = [0.5 * stiff * (com_values - eq_pos)**2 for com_values, eq_pos in zip(bins, x_range)]
    return diffrence

get_window_bias_potential_value(bins, xmin, xmax, n_windows, stiff)


In [ ]:
#I want to 
umbrella_bias = get_bias_potential_value(bins, xmin, xmax, n_windows, stiff)
com_values = list(com_by_window.values())

In [ ]:
umbrella_bias = get_bias_potential_value(bins, xmin, xmax, n_windows, stiff)
umbrella_bias

In [ ]:
master_H = np.zeros(199)

for (H, bins), free, bias in zip(histograms, free_shift, umbrella_bias):
    # print(H)
    # H = H * bias
    # print(H)
    H = np.array([-np.log(count) if count > 0 else 0 for count in H])
    H = H - bias + free
    # print(H)
    # H = np.array([np.exp(-count) for count in H])
    # H = np.array([count if count != 1 else 0 for count in H ])
    print(H)
    master_H += H

master_H = master_H

In [ ]:
master_H /= sum(master_H)
weight_avg = bins[1:] * master_H

In [ ]:
sum(weight_avg) * 0.85

In [ ]:
weight_avg

In [ ]:
plt.plot(bins[1:] * 0.85, master_H)

In [ ]:
for (H, bins), bias in zip(histograms, biases):
    plt.plot(bins[1:], -np.log(H) * bias)

In [ ]:
master_H

In [ ]:
H

In [ ]:
n_data = 50000

n_chunks = 2
data_added_per_iteration = 2

wham_dir = os.path.abspath('/scratch/mlsample/ipy_oxDNA/wham/wham')
n_bins = '200'
tol = '1e-5'
n_boot = '0'
xmin = 0
xmax = 5
n_windows = 46

stiff = 0.8

us.convergence_analysis(n_chunks, data_added_per_iteration, wham_dir, xmin, xmax, stiff, n_bins, tol, n_boot)

In [ ]:
us.wham.plot_truncated_free_energy()
plt.legend()
us.wham.plot_chunks_free_energy()
plt.legend()

<font size='8'><a class="anchor" id="3"></a> <h2 align="center"> <font color='black'>Convergence analysis</font></h2>

In [ ]:
maxes =  [free_energy['Free'] for free_energy in us.chunk_dirs_free]

In [ ]:
for us in us_list:
    us.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
free = pd.read_csv(f'{us.system_dir}/production/com_dir/freefile', sep='\t')

In [ ]:
free_vals = free.iloc[-95:].Free

In [ ]:
free_vals.iloc[1:]

In [ ]:
H_vals = np.zeros(94)
us.hb_list_observable(print_every=1e3)
for idx, sim in enumerate(us.production_sims):
    sim_dir = sim.sim_dir
    file_name = us.observables_list[1]['output']['name']
    df = pd.read_csv(f"{sim_dir}/{file_name}", header=None)
    df = np.concatenate(np.array(df))
    H, bins = np.histogram(df, density=False, bins=np.linspace(0, 8, 95))
    H_vals += H

In [ ]:
prob_H = H_vals / sum(H_vals)

In [ ]:
plt.plot(bins[:-1], -np.log(prob_H))

In [ ]:
file_name = us.observables_list[0]['output']['name']